In [1]:
import numpy as np
import pandas as pd

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff

## Importar datos

In [2]:
    # mi DataFrame
compras = pd.read_csv("/home/dog/AnacondaProjects/data_mining_course/compras.csv")
productos = pd.read_csv("/home/dog/AnacondaProjects/data_mining_course/prods.csv")

    # hacemos un join
master = pd.merge(compras, productos, left_on="codigo", right_on="ID_producto")

In [ ]:
    # parseado de fechas a objeto datetime
master["fecha"] = pd.to_datetime(master["fecha"])

    # creamos una nueva columna donde sacamos el mes de 
master["mes"] = master["fecha"].dt.month

In [3]:
    # crear columna de precio total
master["precio_total"] = master["precio"] * master["unidades"]

In [4]:
    # numero de ticket con mayor facturacion
ticket_grande = master.groupby(by=["nro_ticket"])["precio_total"].agg("sum").sort_values(ascending=False).keys()[0]

In [7]:
    # groupby por seccion
secciones_ticket_grande = master.loc[master["nro_ticket"]==ticket_grande].groupby(by=["Seccion"])["precio"].agg("sum").sort_values(ascending=False)

### Funcion personalizada de un grafico de torta (pie chart)

In [8]:
def torta_personalizada(groupby_pandas, titulo_grafico):
    """
    Tenemos en cuenta que un groupby funciona como un diccionario donde el indice
    es el valor por el cual agrupamos y los valores son el resultado del groupby.
    """
        # sacar datos del groupby
    labels = groupby_pandas.keys()
    values = groupby_pandas.get_values()
    
        # crear nuestra torta
    calcado = go.Pie(labels=labels, 
                     values=values,
                     textposition="outside", # posicion del texto
                     textinfo = "label+percent") # que datos queremos (etiqueta + porcentaje))

    data = [calcado]
    
        # especificamos caracteristicas del grafico
    layout = go.Layout(height = 600, # altura del grafico
                       width = 1300, # largo del grafico
                       autosize = False,
                       title = str(titulo_grafico), # segundo argumento: titulo del grafico
                       legend=dict(orientation="v", 
                                   font=dict(size=30)
                                   )
                       )
        
        # juntamos todo
    fig = go.Figure(data = data, layout = layout)

        # return el grafico hecho
    plotly.offline.plot(fig, show_link = False)

In [9]:
    # invocar la funcion que acabamos de crear
torta_personalizada(secciones_ticket_grande, "Gasto por seccion del ticket mas grande")

In [15]:
    # podemos crear cualquier tipo de funcion siempre y cuando sabemos como acceder a los elementos
def buscar_ticket(numero_ticket, dataframe):
    return dataframe.loc[dataframe["nro_ticket"]==numero_ticket]

In [16]:
buscar_ticket(2222, master)

,Unnamed: 0,nro_ticket,codigo,cupon,unidades,precio,fecha,hora_venta,cantidad_cupon,ID_producto,Descripcion_producto,Division,Seccion,Sub_sección,Categoria,Código_Barra,Marca,precio_total
404176,1857552,2222,8005,0,1.0,4250.0,2017-11-01 00:00:00.000,1341,1,8005,GALLETA C/ ANIS X KG.,Perecederos,Panadería,Panadería,Frescos,8800010111589,LORENZITO,4250.0
418297,1857554,2222,80613,0,2.0,3500.0,2017-11-01 00:00:00.000,1341,1,80613,EMP. DE POLLO AL HORNO,Perecederos,Rotiseria,Listado de Menu,Minutas,8800010189533,LORENZITO,7000.0
979970,2762455,2222,9099176,0,1.0,4400.0,2017-11-02 00:00:00.000,942,1,9099176,LACTOLANDA LECHE SACHET VITAL LIGHT DE 1LT,Perecederos,Lacteos,Leche,Sachet,7840042000377,Lactolanda,4400.0
2950863,2762454,2222,9064680,0,1.0,1950.0,2017-11-02 00:00:00.000,942,1,9064680,MENTHOPLUS MOUNTAIN BLAS 12X12X30,Almacen,Golosinas,Caramelos,Duros,77972743,MENTHO PLUS,1950.0
3369696,1857553,2222,425297,0,1.0,14900.0,2017-11-01 00:00:00.000,1341,1,425297,COFLER CHOC.LECH 4*16*170GR,Almacen,Golosinas,Chocolate en barra,Leche,7790580103583,COFLER,14900.0
3455517,1857555,2222,282033,0,1.0,2450.0,2017-11-01 00:00:00.000,1341,1,282033,MIRINDA GUARANA 330 CC DESCARTABLE,Bebidas,Sin Alcohol,Gaseosas,Descartable,7840028202115,MIRINDA,2450.0


In [17]:
    # barras agrupadas original
calcado1 = go.Bar(x=["Universitario", "Graduado", "Sin estudio superior"], y=[4,5,6], name="Junio")
calcado2 = go.Bar(x=["Universitario", "Graduado", "Sin estudio superior"], y=[7,2,5], name="Julio")
calcado3 = go.Bar(x=["Universitario", "Graduado", "Sin estudio superior"], y=[3,6,7], name="Agosto")

data = [calcado1, calcado2, calcado3]

layout = go.Layout(barmode="group",
                   title="Ventas por mes")

plotly.offline.plot({
    "data": data,
    "layout": layout
})

'file:///home/dog/AnacondaProjects/data_mining_course/temp-plot.html'

In [79]:
#secciones = ["Carniceria", "Verduleria", "Alcoholicas"]
#nombres = ["Junio", "Julio", "Agosto"]
#dataframe = master

#barras_agrupadas(secciones, meses, nombres_meses, master)

def barras_agrupadas(lista_secciones, lista_nombres, lista_meses, dataframe):
    """
    Esta es una funcion, que crea un minidataframe. Soy una docstring
    y aparezco cuando me llaman con help(barras_agrupadas)
    """
    
    # separar 3 dataframes de acuerdo a las secciones
    dataframe_secciones = list()
    
    for i in range(len(lista_secciones)):
        dataframe_secciones.append(dataframe.loc[dataframe["Seccion"]==lista_secciones[i]])
    
    # volver a separar los 3 dataframes en 3 dataframes (9 en total) de acuerdo a los meses
    subsecciones_mes = list()
    
    for l in range(len(dataframe_secciones)):
        for k in range(len(dataframe_secciones)):
            subsecciones_mes.append(dataframe_secciones[k].loc[dataframe_secciones[k]["mes"]==lista_nombres[l]]["precio_total"].sum())
    
    calcado1 = go.Bar(x=lista_secciones, y=subsecciones_mes[0:3], name=lista_meses[0])
    calcado2 = go.Bar(x=lista_secciones, y=subsecciones_mes[3:6], name=lista_meses[1])
    calcado3 = go.Bar(x=lista_secciones, y=subsecciones_mes[6:9], name=lista_meses[2])
    
    data = [calcado1, calcado2, calcado3]

    layout = go.Layout(barmode="group",
                   title="Ventas por mes")

    return plotly.offline.plot({
        "data": data,
        "layout": layout
    })

In [80]:
    # ejecutamos nuestra funcion personalizada
barras_agrupadas(["Carniceria", "Verdulería", "Alcoholicas"],
                 [8, 9, 10], 
                 ["Agosto", "Septiembre", "Octubre"],
                 master)

'file:///home/dog/AnacondaProjects/data_mining_course/temp-plot.html'

In [73]:
    # podemos chequear el docstring de nuestras funciones llamando el comando help()
help(barras_agrupadas)

Help on function barras_agrupadas in module __main__:

barras_agrupadas(lista_secciones, lista_nombres, dataframe)
    Esta es una funcion, que crea un minidataframe



## Ejercicios varios

In [85]:
    # sub secciones de bebidas con alcohol
master.loc[master["Seccion"]=="Alcoholicas"]["Sub_sección"].unique()

array(['Vinos', 'Cañas', 'Cervezas', 'Espumantes', 'Vodka', 'Whisky',
       'Aperitivos', 'Tequila', 'Ron', 'Licor', 'Martini', 'Gin', 'Coñac',
       'Pisco', 'Ginebra', 'Fernet', 'Sangrias'], dtype=object)

In [86]:
    # nombre de la categoria es Lacteos (sin acento)
master.loc[master["Seccion"].str.contains("Lact")] # Lacteos

,Unnamed: 0,nro_ticket,codigo,cupon,unidades,precio,fecha,hora_venta,cantidad_cupon,ID_producto,Descripcion_producto,Division,Seccion,Sub_sección,Categoria,Código_Barra,Marca,precio_total,mes
77402,3,134,2020,0,4.0,1400.0,2017-08-01,733,1,2020,LACTOLANDA YOGULAC FRUTILLA POTE 140 G,Perecederos,Lacteos,Yogurt,Pote,7840042100633,Lactolanda,5600.0,8
77403,449,1358,2020,0,1.0,1400.0,2017-08-01,1023,1,2020,LACTOLANDA YOGULAC FRUTILLA POTE 140 G,Perecederos,Lacteos,Yogurt,Pote,7840042100633,Lactolanda,1400.0,8
77404,2650,155246,2020,0,3.0,1400.0,2017-09-07,1629,1,2020,LACTOLANDA YOGULAC FRUTILLA POTE 140 G,Perecederos,Lacteos,Yogurt,Pote,7840042100633,Lactolanda,4200.0,9
77405,10203,1135235,2020,0,4.0,1400.0,2017-09-07,1400,1,2020,LACTOLANDA YOGULAC FRUTILLA POTE 140 G,Perecederos,Lacteos,Yogurt,Pote,7840042100633,Lactolanda,5600.0,9
77406,11421,1144160,2020,0,3.0,1400.0,2017-09-07,932,1,2020,LACTOLANDA YOGULAC FRUTILLA POTE 140 G,Perecederos,Lacteos,Yogurt,Pote,7840042100633,Lactolanda,4200.0,9
77407,13769,1183565,2020,0,1.0,1400.0,2017-09-07,729,1,2020,LACTOLANDA YOGULAC FRUTILLA POTE 140 G,Perecederos,Lacteos,Yogurt,Pote,7840042100633,Lactolanda,1400.0,9
77408,16313,155378,2020,0,2.0,1300.0,2017-09-08,1737,1,2020,LACTOLANDA YOGULAC FRUTILLA POTE 140 G,Perecederos,Lacteos,Yogurt,Pote,7840042100633,Lactolanda,2600.0,9
77409,17310,175166,2020,0,3.0,1300.0,2017-09-08,1608,1,2020,LACTOLANDA YOGULAC FRUTILLA POTE 140 G,Perecederos,Lacteos,Yogurt,Pote,7840042100633,Lactolanda,3900.0,9
77410,20985,1104269,2020,0,3.0,1400.0,2017-09-08,1018,1,2020,LACTOLANDA YOGULAC FRUTILLA POTE 140 G,Perecederos,Lacteos,Yogurt,Pote,7840042100633,Lactolanda,4200.0,9
77411,23200,1115993,2020,0,8.0,1300.0,2017-09-08,2115,1,2020,LACTOLANDA YOGULAC FRUTILLA POTE 140 G,Perecederos,Lacteos,Yogurt,Pote,7840042100633,Lactolanda,10400.0,9


In [88]:
master.loc[master["Seccion"]=="Sin Alcohol"]

,Unnamed: 0,nro_ticket,codigo,cupon,unidades,precio,fecha,hora_venta,cantidad_cupon,ID_producto,Descripcion_producto,Division,Seccion,Sub_sección,Categoria,Código_Barra,Marca,precio_total,mes
80239,8,135,482865,0,1.0,7900.0,2017-08-01,743,1,482865,COCA COLA GASEOSA 1.5 LT N.R *6.,Bebidas,Sin Alcohol,Gaseosas,Descartable,7840058000675,COCA COLA,7900.0,8
80240,2722,155257,482865,0,1.0,7900.0,2017-09-07,1819,1,482865,COCA COLA GASEOSA 1.5 LT N.R *6.,Bebidas,Sin Alcohol,Gaseosas,Descartable,7840058000675,COCA COLA,7900.0,9
80241,3263,155314,482865,0,1.0,7900.0,2017-09-07,2018,1,482865,COCA COLA GASEOSA 1.5 LT N.R *6.,Bebidas,Sin Alcohol,Gaseosas,Descartable,7840058000675,COCA COLA,7900.0,9
80242,4588,163588,482865,0,1.0,7900.0,2017-09-07,1238,1,482865,COCA COLA GASEOSA 1.5 LT N.R *6.,Bebidas,Sin Alcohol,Gaseosas,Descartable,7840058000675,COCA COLA,7900.0,9
80243,5356,175110,482865,0,1.0,7900.0,2017-09-07,1946,1,482865,COCA COLA GASEOSA 1.5 LT N.R *6.,Bebidas,Sin Alcohol,Gaseosas,Descartable,7840058000675,COCA COLA,7900.0,9
80244,8484,1115738,482865,0,1.0,7900.0,2017-09-07,1821,1,482865,COCA COLA GASEOSA 1.5 LT N.R *6.,Bebidas,Sin Alcohol,Gaseosas,Descartable,7840058000675,COCA COLA,7900.0,9
80245,9296,1115812,482865,0,1.0,7900.0,2017-09-07,2131,1,482865,COCA COLA GASEOSA 1.5 LT N.R *6.,Bebidas,Sin Alcohol,Gaseosas,Descartable,7840058000675,COCA COLA,7900.0,9
80246,9348,1115823,482865,0,1.0,7900.0,2017-09-07,2156,1,482865,COCA COLA GASEOSA 1.5 LT N.R *6.,Bebidas,Sin Alcohol,Gaseosas,Descartable,7840058000675,COCA COLA,7900.0,9
80247,17327,175166,482865,0,1.0,7900.0,2017-09-08,1608,1,482865,COCA COLA GASEOSA 1.5 LT N.R *6.,Bebidas,Sin Alcohol,Gaseosas,Descartable,7840058000675,COCA COLA,7900.0,9
80248,18006,175226,482865,0,1.0,7900.0,2017-09-08,2028,1,482865,COCA COLA GASEOSA 1.5 LT N.R *6.,Bebidas,Sin Alcohol,Gaseosas,Descartable,7840058000675,COCA COLA,7900.0,9


In [89]:
secciones = ["Alcoholicas", "Lacteos", "Sin Alcohol"]
meses_numero = [8, 9, 10]
meses_nombre = ["Agosto", "Septiembre", "Octubre"]

barras_agrupadas(secciones, meses_numero, meses_nombre, master)

'file:///home/dog/AnacondaProjects/data_mining_course/temp-plot.html'

In [92]:
bebidas_alcoholicas = master.loc[master["Seccion"]=="Alcoholicas"].groupby(by=["Sub_sección"])["precio_total"].agg("sum").sort_values(ascending=False)

torta_personalizada(bebidas_alcoholicas, "Porcentages de bebidas alcoholicas vendidas")

In [95]:
bebidas_sin_alcohol = master.loc[(master["Seccion"]=="Sin Alcohol")
                                 & (master["Sub_sección"]=="Gaseosas")].groupby(by=["Marca"])["precio_total"].agg("sum").sort_values(ascending=False)

torta_personalizada(bebidas_sin_alcohol, "Market share gaseosas")

In [107]:
    # crear una nueva columna con NULL (None es el NULL de python)
master["marca_master"] = None

    # reemplazar la nueva columna con strings si es que se cumple la condicion
master.loc[(master["Marca"].str.contains("COCA COLA|FANTA|SPRITE")), ["marca_master"]] = "Coca Cola"
master.loc[(master["Descripcion_producto"].str.contains("PEPSI|MIRINDA|7UP")), ["marca_master"]] = "Pepsi"

In [110]:
    # graficamos nuestra nueva columna
coca_vs_pepsi = master.groupby(by=["marca_master"])["precio_total"].agg("sum").sort_values(ascending=False)

torta_personalizada(coca_vs_pepsi, "Coca Cola vs Pepsi")

In [ ]:
    # si no queremos volver a hacer todo este proceso, podemos usar .to_csv() para crear un nuevo archivo
master.to_csv("/home/dog/AnacondaProjects/master.csv")